<a href="https://colab.research.google.com/github/Arwa601/Projet_Big_Data_Spark/blob/main/arwa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer, PCA, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("Movies Metadata Analysis") \
    .getOrCreate()

# Load the CSV file into a Spark DataFrame
df = spark.read.csv('/content/movies_metadata.csv', header=True, inferSchema=True)

# Show initial data and summary statistics
df.show(5)
df.describe().show()

# Identify numeric columns
numeric_columns = [col for col in df.columns if dict(df.dtypes)[col] in ['int', 'float']]

# Convert categorical columns to numeric
categorical_columns = ['adult', 'belongs_to_collection', 'genres', 'homepage', 'original_language', 'production_companies', 'release_date']  # Adjust this list based on your dataset
indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_index", handleInvalid="skip").fit(df) for col in categorical_columns]

# Apply the indexers to the DataFrame
for indexer in indexers:
    df = indexer.transform(df)

# Combine numeric and indexed categorical columns
feature_columns = numeric_columns + [f"{col}_index" for col in categorical_columns]

# Assemblage des colonnes de caractéristiques
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_features = assembler.transform(df)




# Standardize the features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
scaler_model = scaler.fit(df_features)
df_scaled = scaler_model.transform(df_features)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------+--------------------+-----------------+
|adult|belongs_to_collection|  budget|              genres|            homepage|   id|  imdb_id|original_language|      original_title|            overview|popularity|         poster_path|production_companies|production_countries|        release_date|             revenue|             runtime|    spoken_languages|  status|             tagline|               title|   video|        vote_average|       vote_count|
+-----+---------------------+--------+-----

In [ ]:
from pyspark.ml.feature import PCA

# Configure PCA
pca = PCA(k=2, inputCol="features", outputCol="pca_features")
model_pca = pca.fit(df_features)
df_pca = model_pca.transform(df_features)

# Show the PCA results
df_pca.select("pca_features").show(5)

+--------------------+
|        pca_features|
+--------------------+
|[-3408.5070720627...|
|[-2833.5842082628...|
|[-1064.2246243056...|
|[4719.47256976112...|
|[-165.29169082337...|
+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import StringIndexer

# Index the label column (adjust 'your_label_column' to your actual label column name)
label_indexer = StringIndexer(inputCol="vote_count", outputCol="label").fit(df_pca)
df_pca = label_indexer.transform(df_pca)

In [ ]:
print(df_pca.columns)

['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'adult_index', 'belongs_to_collection_index', 'genres_index', 'homepage_index', 'original_language_index', 'production_companies_index', 'release_date_index', 'features', 'pca_features', 'label']


In [ ]:
from pyspark.ml.classification import LogisticRegression

# Define the logistic regression model
lr = LogisticRegression(featuresCol="pca_features", labelCol="label", regParam=0.01)

# Train the model
model_lr = lr.fit(df_pca)

Py4JJavaError: An error occurred while calling o5151.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 527.0 failed 1 times, most recent failure: Lost task 1.0 in stage 527.0 (TID 737) (3112c7148bcf executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`StringIndexerModel$$Lambda$3766/0x00000008415ca840`: (string) => double).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.ha

In [ ]:
# Make predictions
predictions = model_lr.transform(df_pca)

# Show predictions results
predictions.select("label", "prediction", "probability").show(5)

# Initialize the evaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

# Compute the accuracy
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy = {accuracy:.2f}")

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|  8.0|       0.0|[0.29959029794299...|
|110.0|       0.0|[0.29940873356874...|
| 31.0|       0.0|[0.41343060901944...|
| 40.0|       0.0|[0.35749079583758...|
|173.0|       0.0|[0.46995571218896...|
+-----+----------+--------------------+
only showing top 5 rows

Test Accuracy = 0.35


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialize the evaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

# Compute the accuracy
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy = {accuracy:.2f}")

Test Accuracy = 0.35
